In [50]:
import numpy as np
import pandas as pd
import scipy
import pandas_datareader as web
import seaborn as sns
import yfinance as yf

# Plotting
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab

#Statistical calculation
from scipy.stats import norm
from scipy.optimize import minimize, Bounds

byddata = pd.read_csv('BYD.csv')
cocodata = pd.read_csv('COCO.csv')
elfdata = pd.read_csv('ELF.csv')
kodata = pd.read_csv('KO.csv')
sdata = pd.read_csv('S.csv')
spxdata = pd.read_csv('SPX.csv')

byddata.rename(columns={'Close': 'bydprice'}, inplace=True)
cocodata.rename(columns={'Close': 'cocoprice'}, inplace=True)
elfdata.rename(columns={'Close': 'elfprice'}, inplace=True)
kodata.rename(columns={'Close': 'koprice'}, inplace=True)
sdata.rename(columns={'Close': 'sprice'}, inplace=True)
spxdata.rename(columns={'Close': 'spxprice'}, inplace=True)

byd = byddata['bydprice']
coco = cocodata['cocoprice']
elf = elfdata['elfprice']
ko = kodata['koprice']
s = sdata['sprice']
spx = spxdata['spxprice']

In [55]:
# Import packages
import numpy as np
import pandas as pd
from scipy.optimize import minimize, Bounds



# The function to calculate the portfolio volatility is as follows
# Var = w' * VCV * w
# Std = sqrt(Var)

def portfolio_volatility(weights, covmat):
    w = np.array(weights)
    covmat = np.asarray(covmat)
    return np.dot(np.dot(w.T, covmat), weights) ** 0.5
# We first need to define the risk contribution function, which measures the % contribution of risk of each asset to the total portfolio volatility

def risk_contribution(weights: pd.DataFrame, covmat: pd.DataFrame):
    """
    Computes the contribution to risk of the constituents of a portfolio

    Parameters
    ----------
    w: portfolio weights
    covmat: covariance matrix

    Returns
    -------
    Contribution to risk of each constituent

    """
    w = np.array(weights).reshape(-1, 1)
    cov = np.asarray(covmat)

    port_var = portfolio_volatility(w, cov) ** 2
    # Marginal contribution to risk
    mcr = np.dot(cov, w)
    # Percentage contribution to risk     
    cr = np.multiply(mcr, w) / port_var
    
    return cr
# Now, we have to define the function to be optimized.
# This is the function we are going to feed the optimizer with

def risk_budjet_error(weights, covmat):
    perc_risk_cont = risk_contribution(weights, covmat).ravel()
    n_assets = covmat.shape[0]
    risk_target = np.array([1 / n_assets] * n_assets)  # The target risk for each asset is equal to the portfolio risk / n of assets
    j = sum(np.square(risk_target - perc_risk_cont))  # sum of squared errors
    return j
# The optimization function

def optimize(func, covmat, target_return=None, allow_short=False):
    init_weights = [1 / covmat.shape[0]] * covmat.shape[0]  # initialize weights array

    opt_bounds = Bounds(0, 1) if not allow_short else None  # Constraint on short selling

    opt_constraints = {'type': 'eq',
                       'fun': lambda w: 1.0 - np.sum(w)}  # the weights need to sum up to 100%
    
    if target_return is not None:  # add a constraint to meet a specific target return. Default as False as it might result in no solutions 
                                   # to the optimization problem
        opt_constraints = ({'type': 'eq',
                            'fun': lambda w: 1.0 - np.sum(w)},
                           {'type': 'eq',
                            'fun': lambda w: target_return - w.T @ exp_ret})

    optimal_weights = minimize(func,
                               init_weights,
                               args=(covmat),
                               method='SLSQP',
                               bounds=opt_bounds,
                               constraints=opt_constraints)

    return optimal_weights.x

In [62]:
newdataframe = pd.concat([byd,coco,elf, ko,s], axis=1) # merge datasets
returns = newdataframe.pct_change().dropna() #simple returns
FinalDataFrame = returns.loc[(returns!=0).any(axis=1)]

vcv = returns.cov() #variance covariance
vcv_ann = vcv * np.sqrt(252)

risk_parity_weights = optimize(risk_budjet_error,vcv_ann)
print(risk_parity_weights)

risk_parity_weights = pd.DataFrame(risk_parity_weights, index=vcv_ann.index, columns=["Weight"])
risk_parity_weights

risk_contribution(risk_parity_weights, vcv_ann)

p_vol = round(portfolio_volatility(risk_parity_weights, vcv_ann)[0][0] * 100,2)

print("Portfolio (ex-ante) annualized volatility is: {v}%".format(v=p_vol))
print("Single constituent's volatility is: {v}".format(v=round(p_vol / 3, 2)))





[0.17191521 0.13620398 0.14523945 0.459502   0.08713936]
Portfolio (ex-ante) annualized volatility is: 5.13%
Single constituent's volatility is: 1.71
